# **NeuralProphet**

Documentation [here](https://ourownstory.github.io/neural_prophet/changes-from-prophet/)

In [ ]:
# installing neuralprophet lib 

!pip install neuralprophet

In [ ]:
import pandas as pd
import numpy as np

from neuralprophet import NeuralProphet
from neuralprophet.plot_forecast import plot_components

import plotly.express as px
import plotly.graph_objects as go


In [ ]:
def getFloat(start,stop):
    return start + np.random.sample(1) * stop

In [ ]:
database = pd.DataFrame()
for y in range(2000, 2021):
  for m in range(1,13):
    for d in range(1,29):
        vvt = 0
        if m < 3:     # jan e fev 
          i = getFloat(500, 700)[0]
        elif m == 6:  # jun 
          i = getFloat(1500,2000)[0]
        elif m == 12: # dez 
          i = getFloat(1000,1500)[0]
        else:         # restante
          i = getFloat(800, 1000)[0]
        if y % 2 == 0 and m < 7:
          i += i*0.1
          vvt = 1
        s = {'ds': f'{y}-{m}-{d}', 'y': i, 'vvt': vvt}
        database = database.append(s, ignore_index=True)

In [ ]:
database['ds'] = database.ds.astype('datetime64[ns]')

In [ ]:
import plotly.graph_objects as go
goTemplates = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=database.ds, y=database.y,
                    mode='lines+markers',
                    name='total vendas'))

fig.update_layout(template=goTemplates[1], title="Histórico de Vendas")

In [ ]:
database=database.drop(['vvt'], axis=1)

In [ ]:
m = NeuralProphet(daily_seasonality=True)
metrics = m.fit(database, freq="M")
metrics.head(3)

In [ ]:
future = m.make_future_dataframe(database, periods=13, n_historic_predictions=len(database))
forecast = m.predict(future)

In [ ]:
fig = go.Figure()
predict = forecast[forecast.ds.dt.year>2020]
fig.add_trace(go.Scatter(x=database.ds, y=database.y,
                    mode='lines+markers',
                    name='Realizado'))
fig.add_trace(go.Scatter(x=predict.ds, y=predict.yhat1,
                    mode='lines+markers',
                    name='Previsao'))
fig.update_layout(template=goTemplates[1], title="ML - Previsao de vendas")

In [ ]:
m.plot(forecast)

In [ ]:
plot_components(m, forecast)

In [ ]:
m.plot_components(predict, residuals=True)

In [ ]:
m.plot_parameters()